In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from typing import Callable

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

np.random.seed(42)

In [2]:
def sample_next_state(index: int, P: np.ndarray) -> int:
    probs = P[index]

    next = np.random.choice(range(P.shape[1]), p=probs)

    return int(next)

In [3]:
def td_loop(
        n_iter: int, 
        X: np.ndarray, 
        y: np.ndarray, 
        P: np.ndarray,
        link: Callable[[np.ndarray], np.ndarray], 
        inv_link: Callable[[np.ndarray], np.ndarray], 
        gamma: float, 
        alpha: float,
        epsilon: float,
    ) -> np.ndarray:
    n_samples = X.shape[0]
    n_features = X.shape[1]

    w = np.zeros(n_features)

    curr_index = int(np.random.randint(n_samples))
    curr_x = X[curr_index]
    curr_y = y[curr_index]

    i = 0
    grad = np.ones_like(w) * np.inf

    while i < n_iter and np.linalg.norm(alpha * grad, 2) > epsilon:
        # Next state samples
        next_index = sample_next_state(index=curr_index, P=P)
        next_x = X[next_index]
        next_y = y[next_index]

        # Find predictions
        curr_z = np.dot(curr_x, w)
        next_z = np.dot(next_x, w)

        # Find rewards
        r = inv_link(curr_y) - gamma * inv_link(next_y)

        # TD target
        z_t = r + gamma * next_z

        # Find gradient
        grad = (link(curr_z) - link(z_t)) * curr_x

        # Update weights
        w -= alpha * grad
        
        # Update state and index
        curr_index, curr_x, curr_y = next_index, next_x, next_y
        i += 1

    return w

In [4]:
# Generate synthetic data
num_samples = 100
num_features = 3

X = np.random.randn(num_samples, num_features)
true_w = np.array([2.0, -3.5, 1.0])
y = X @ true_w + np.random.randn(num_samples) * 0.1  # Adding noise

alpha = 0.01  # Learning rate
gamma = 0.9   # Discount factor
num_iterations = 1e6  # Number of iterations
epsilon = 1e-7

P = np.ones((num_samples, num_samples)) / num_samples # Equal probability to move to any state

In [5]:
w_hat = td_loop(
    n_iter=num_iterations,
    X=X,
    y=y,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
)

error = np.linalg.norm(w_hat - true_w, 2)

print(w_hat)
print(f'Error: {error:.5f}')

[ 2.00494837 -3.50096249  0.99077277]
Error: 0.01051


In [6]:
housedata = np.loadtxt('data\\readyhousedata.txt', delimiter=',')

X = housedata[:, :-1]
y = housedata[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression() 
reg.fit(X_train, y_train)

weights = reg.coef_
intercept = reg.intercept_

print(weights)
print("Intercept:", intercept)

score = reg.score(X_test, y_test)
print("Model R^2 score:", score)

[-10.99366542   5.00578355   0.63581661   2.44872684 -10.31186138
  17.51763825   1.03114679 -17.77574728   7.34087816  -6.42763345
  -8.79966234   3.03563353 -20.51935608]
Intercept: 29.04431567479631
Model R^2 score: 0.7730570034661727


In [13]:
num_samples = X_train.shape[0]
P = np.ones((num_samples, num_samples)) / num_samples # Equal probability to move to any state

alpha = 0.01  # Learning rate
gamma = 0   # Discount factor
num_iterations = 1e6  # Number of iterations
epsilon = 1e-9

w_hat_house = td_loop(
    n_iter=num_iterations,
    X=X_train,
    y=y_train,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
)

print(w_hat_house)

[ -8.87567805   5.97776876   4.21570583   2.62107647  -4.21122996
  34.5166922    3.39308524  -5.43650536   6.17110026  -6.18122986
  -3.72481184   9.43856777 -12.38355551]


In [16]:
pred_TD = np.dot(X_test, w_hat_house)
pred_L2 = np.dot(X_test, weights)

rmse_TD = np.sqrt(mean_squared_error(y_test, pred_TD))
rmse_L2 = np.sqrt(mean_squared_error(y_test, pred_L2))

print("RMSE on the test set using TD:", rmse_TD)
print("RMSE on the test set using L2 Regression:", rmse_L2)
print("---------------")
print("Norm of weights from TD:", np.linalg.norm(w_hat_house, 2))
print("Norm of weights from L2 Regression:", np.linalg.norm(weights, 2))
print("Norm of difference in weights:", np.linalg.norm(weights - w_hat_house, 2))


RMSE on the test set using TD: 4.228110329436598
RMSE on the test set using L2 Regression: 29.04652675774383
---------------
Norm of weights from TD: 41.49653029749279
Norm of weights from L2 Regression: 38.54163631701667
Norm of difference in weights: 25.232303509220113
